In [ ]:
'''
grand average plots based on pile_all mat files generated by compare_constructs_GCaMP96uf
'''

from scipy.io import loadmat
import numpy as np
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

ap_idx = 0 # 0: 1AP, 1: 3AP, 2:10AP, 3:160AP

plot_subset_for_paper = 1 # 0: plot entire dataset, save as html. 1: plot subset, save as pdf for paper
html_write_dir = r'D:\site\ilyakolb.github.io\interactive_norm_plots.html'
pdf_dir = r"D:\ufgcamp_paper_data\culture-screen-figs/norm_plots.pdf"
plot_mat = loadmat(r'data/unnormPlots_singleWells_struct.mat')

subplot_titles = ['peak dF/F', 'SNR', 'half-rise time (ms)', 'full rise time (ms)', 'half-decay time (ms)']
df_cols = subplot_titles.copy()
df_cols.insert(0,'construct')

# labels and order of legend
if plot_subset_for_paper:
    hits_label = ['jGCaMP8f', 'jGCaMP8m', 'jGCaMP8s', 'GCaMP6s', 'jGCaMP7f', 'jGCaMP7s', 'XCaMP-Gf']# [h[0] for h in hits[0]]
else:
    hits_label = ['jGCaMP8f', 'jGCaMP8m', 'jGCaMP8s','jGCaMP8.712', 'GCaMP6s', 'GCaMP6f', 'jGCaMP7f', 'jGCaMP7s', 
                'jGCaMP7c', 'jGCaMP7b', 'XCaMP-Gf', 'XCaMP-G', 'XCaMP-Gf0']# [h[0] for h in hits[0]]

# magnify the kinetics of the following hits
magnify_hits_label = ['jGCaMP8f', 'jGCaMP8m', 'jGCaMP8s']

all_data = plot_mat['unnormPlots_singleWells_struct'][0]

all_constructs = [c[0] for c in all_data['construct']]

# create dataframe
df = pd.DataFrame(columns=df_cols)

for construct_idx in range(len(all_constructs)):
    n_wells = len(all_data[construct_idx][1][ap_idx])
    n_cells = all_data[construct_idx][1].shape[1] # TEMPORARY BUGFIX
    
    dff = all_data[construct_idx][1][ap_idx]
    snr = all_data[construct_idx][2][ap_idx] #[:n_cells] # TEMPORARY SNR BUGFIX
    
    # get kinetics, convert to milliseconds
    halfrise = all_data[construct_idx][3][ap_idx]*1000
    fullrise = all_data[construct_idx][4][ap_idx]*1000
    halfdecay = all_data[construct_idx][5][ap_idx]*1000
    
    current_construct = [all_data[construct_idx][0][0]] * n_wells
    df_construct = pd.DataFrame(np.array([current_construct, dff, snr, halfrise, fullrise, halfdecay]).T, columns=df_cols)
    df = df.append(df_construct)

# cast data columns as floats
castdict = {x: 'float' for x in subplot_titles}
df = df.astype(castdict)
print('Saving...', end='')
df.to_pickle('data/grand-avg-data_{}.pkl'.format(ap_idx))
print('Done')

In [ ]:
'''
plot grand averages, single wells
if no re-processing is needed, do not run previous cell

@todo: save figs
@todo: export csvs for prism

'''
all_idx = [0,1,2,3] # all APs to plot
all_idx_str = ['1AP', '3AP', '10AP', '160AP']

for apidx in all_idx:
    print('============{}============'.format(all_idx_str[apidx]))
    df_loaded = pd.read_pickle('data/grand-avg-data_{}.pkl'.format(apidx))
    df_hits = df_loaded[df_loaded.construct.isin(hits_label)]
    df_magnify_hits = df_loaded[df_loaded.construct.isin(magnify_hits_label)]

    f = plt.figure(figsize=(5,20))
    # ax = sns.violinplot(x="construct", y="half-rise time", data=df_hits, inner=None, scale='width')
    f.suptitle(all_idx_str[apidx])
    sns.set_theme()
    sns.set_style('whitegrid')


    subplot_idx = 1
    for var_to_plot in subplot_titles:
        plt.subplot(5,1,subplot_idx)
        subplot_idx +=1
        ax = sns.stripplot(x="construct",
                           y=var_to_plot, 
                           data=df_hits, 
                           order = hits_label, 
                           size=5,
                          alpha=0.6)
        ax = sns.boxplot(x="construct", 
                         y=var_to_plot, 
                         data=df_hits, 
                         order = hits_label, 
                         showfliers=False, 
                         color='white',
                         linewidth=3,
                         width=.5)
        plt.xticks(rotation=-45)
        if 'time' in var_to_plot and (apidx == 0 or apidx == 1):
            # magnification inset
            ins_ax = inset_axes(ax, width="35%", height="60%", loc=2, borderpad=2)
            ins_ax = sns.stripplot(x="construct",
                                y=var_to_plot, 
                                data=df_magnify_hits, 
                                order = magnify_hits_label, 
                                size=5,
                                alpha=0.6)
            ins_ax = sns.boxplot(x="construct", 
                                y=var_to_plot, 
                                data=df_magnify_hits, 
                                order = magnify_hits_label, 
                                showfliers=False, 
                                color='white',
                                linewidth=3,
                                width=.5)
            ins_ax.set_xlabel(None)
            ins_ax.set_ylabel(None)
            ins_ax.set_xticklabels('')
        
        f.tight_layout()
        plt.savefig('figs/grand_avg_{}.pdf'.format(all_idx_str[apidx]))
    sns.despine()
    plt.show()
    # compute grand average statistics
    means = (df_loaded.groupby('construct').mean()*100).apply(np.round)/100 # just a way to get a reasonable number of sig figs
    stds = (df_loaded.groupby('construct').std()*1000).apply(np.round)/1000
    
    display(means.astype('str')+'+/-'+stds.astype('str'))
    


In [ ]:
# stats
import scipy.stats as ss
import scikit_posthocs as sp

apidx = 0

vars_to_compare = ['peak dF/F', 'SNR', 'half-rise time (ms)', 'full rise time (ms)', 'half-decay time (ms)']
for var_to_compare in vars_to_compare:
    df_loaded = pd.read_pickle('data/grand-avg-data_{}.pkl'.format(apidx))

    # constructs to use for statistics
    stats_subset = ['jGCaMP8f', 'jGCaMP8m', 'jGCaMP8s', 'XCaMP-G', 'XCaMP-Gf', 'XCaMP-Gf0']
    df_stats_subset = df_loaded[df_loaded['construct'].isin(stats_subset)]

    # list of values to compare
    stats_list = [df_stats_subset[df_stats_subset['construct']==c][var_to_compare].to_list() for c in stats_subset]

    #list_stats_subset = [df]
    print('==========={}==========='.format(var_to_compare))
    [_,p] = ss.kruskal(*stats_list)# ss.kruskal(jg8f, jg8m, jg8s, xcg, xcgf, xcgf0)
    print('Kruskal-Wallice test: p = {}'.format(p))
    # Dunn's post-hoc
    display(sp.posthoc_dunn(df_stats_subset, val_col = var_to_compare, group_col='construct', p_adjust='bonferroni'))

In [ ]:
L = [df_stats_subset[df_stats_subset['construct']=='jGCaMP8f'][var_to_compare]]

In [ ]:
stats_list